In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from tqdm import tqdm
import os
# import dask.dataframe as dd
import re
# from nltk.stem import WordNetLemmatizer

In [2]:
# nltk.download('punkt')  # For tokenization
# nltk.download('stopwords')  # For stopword removal
# nltk.download('wordnet')

# lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /home/shahbaz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shahbaz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/shahbaz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
df=pd.read_csv("/home/shahbaz/Desktop/A2/wiki.csv",index_col=False)

In [4]:
df

,ARTICLE_ID,TITLE,SECTION_TITLE,SECTION_TEXT
0,0,Anarchism,Introduction,\n\n\n\n\n\n'''Anarchism''' is a political phi...
1,0,Anarchism,Etymology and terminology,\n\nThe term ''anarchism'' is a compound word ...
2,0,Anarchism,History,\n\n===Origins===\nWoodcut from a Diggers docu...
3,0,Anarchism,Anarchist schools of thought,\nPortrait of philosopher Pierre-Joseph Proudh...
4,0,Anarchism,Internal issues and debates,\nconsistent with anarchist values is a contro...
...,...,...,...,...
4999995,816930,Vergulde Draeck,The wrecking of the Vergulde Draeck,In 1656 the Vergulde Draeck set sail from the ...
4999996,816930,Vergulde Draeck,Arrival in Batavia,On 7 May 1656 approximately nine days after th...
4999997,816930,Vergulde Draeck,Rescue Attempts,A number of rescue attempts was conducted by t...
4999998,816930,Vergulde Draeck,Discovery of the Vergulde Draeck,The Vergulde Draeck was discovered on 13 April...


In [5]:
df = df.drop(columns=['TITLE', 'SECTION_TITLE'])

In [6]:
df.isnull().sum()

ARTICLE_ID         0
SECTION_TEXT    3754
dtype: int64

In [7]:
df=df.dropna()
df.isnull().sum()

ARTICLE_ID      0
SECTION_TEXT    0
dtype: int64

In [9]:
df.to_csv("/home/shahbaz/Desktop/A2/clean1.csv",index=False)

## testing with a subset

In [68]:
df=pd.read_csv("/home/shahbaz/Desktop/A2/sample.csv",index_col=False)
df

,ARTICLE_ID,SECTION_TEXT
0,0,\n\n\n\n\n\n'''Anarchism''' is a political phi...
1,0,\n\nThe term ''anarchism'' is a compound word ...
2,0,\n\n===Origins===\nWoodcut from a Diggers docu...
3,0,\nPortrait of philosopher Pierre-Joseph Proudh...
4,0,\nconsistent with anarchist values is a contro...
...,...,...
4995,535,\n* White House biography \n* \n* Andrew Joh...
4996,536,\n\n\n'''Aleksandr Isayevich Solzhenitsyn''' (...
4997,536,\n===Early years===\nSolzhenitsyn was born in ...
4998,536,Solzhenitsyn died of heart failure near Moscow...


In [69]:
df = df.groupby('ARTICLE_ID')['SECTION_TEXT'].apply(lambda x: ' '.join(x)).reset_index()

# Display the first few rows of the result to verify the operation
df


,ARTICLE_ID,SECTION_TEXT
0,0,\n\n\n\n\n\n'''Anarchism''' is a political phi...
1,1,\n\n\n \n\n\n\n\n'''Autism''' is a neurodevelo...
2,2,\n\nPercentage of diffusely reflected sunlight...
3,3,\n\n\n\n\n\n\nWriting cursive forms of A\n'''A...
4,4,\n\n\n\n\n\n\n'''Alabama''' () is a state in t...
...,...,...
532,532,\n\n\n\n\n'''Alexios V Doukas''' or '''Alexius...
533,533,\n\n\n\n\n'''Alexei Petrovich Romanov''' () ( ...
534,534,\n\n\n\n\n'''Andrew Jackson''' (March 15 1767 ...
535,535,\n\n\n\n\n'''Andrew Johnson''' (December 29 18...


In [23]:
x.to_csv("/home/shahbaz/Desktop/A2/sample.csv",index=False)

In [70]:

df

,ARTICLE_ID,SECTION_TEXT
0,0,\n\n\n\n\n\n'''Anarchism''' is a political phi...
1,1,\n\n\n \n\n\n\n\n'''Autism''' is a neurodevelo...
2,2,\n\nPercentage of diffusely reflected sunlight...
3,3,\n\n\n\n\n\n\nWriting cursive forms of A\n'''A...
4,4,\n\n\n\n\n\n\n'''Alabama''' () is a state in t...
...,...,...
532,532,\n\n\n\n\n'''Alexios V Doukas''' or '''Alexius...
533,533,\n\n\n\n\n'''Alexei Petrovich Romanov''' () ( ...
534,534,\n\n\n\n\n'''Andrew Jackson''' (March 15 1767 ...
535,535,\n\n\n\n\n'''Andrew Johnson''' (December 29 18...


In [71]:
df['SECTION_TEXT'] = df['SECTION_TEXT'].str.lower()

In [72]:
def preprocessing(text):
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'[^\w\s]', '', text)  
    
    tokens = word_tokenize(text)
    filtered_tokens = [lemmatizer.lemmatize(word) for word in tokens if word.lower() not in stop_words and word not in punctuation]
    
    return " ".join(filtered_tokens)

# Apply the function to tokenize and clean 'SECTION_TEXT'
df['TOKENIZED_TEXT'] = df['SECTION_TEXT'].apply(preprocessing)

In [73]:
df

,ARTICLE_ID,SECTION_TEXT,TOKENIZED_TEXT
0,0,\n\n\n\n\n\n'''anarchism''' is a political phi...,anarchism political philosophy advocate selfgo...
1,1,\n\n\n \n\n\n\n\n'''autism''' is a neurodevelo...,autism neurodevelopmental disorder characteriz...
2,2,\n\npercentage of diffusely reflected sunlight...,percentage diffusely reflected sunlight relati...
3,3,\n\n\n\n\n\n\nwriting cursive forms of a\n'''a...,writing cursive form named plural aes first le...
4,4,\n\n\n\n\n\n\n'''alabama''' () is a state in t...,alabama state southeastern region united state...
...,...,...,...
532,532,\n\n\n\n\n'''alexios v doukas''' or '''alexius...,alexios v doukas alexius v ducas december byza...
533,533,\n\n\n\n\n'''alexei petrovich romanov''' () ( ...,alexei petrovich romanov russian tsarevich bor...
534,534,\n\n\n\n\n'''andrew jackson''' (march 15 1767 ...,andrew jackson march june american soldier sta...
535,535,\n\n\n\n\n'''andrew johnson''' (december 29 18...,andrew johnson december july th president unit...


In [74]:
print(df['TOKENIZED_TEXT'].iloc[0])


anarchism political philosophy advocate selfgoverned society based voluntary institution often described stateless society although several author defined specifically institution based nonhierarchical free association anarchism hold state undesirable unnecessary harmful antistatism central anarchism specifically entail opposing authority hierarchical organisation conduct human relation including limited state system anarchism usually considered extreme leftwing ideology much anarchist economics anarchist legal philosophy reflects antiauthoritarian interpretation communism collectivism syndicalism mutualism participatory economics anarchism offer fixed body doctrine single particular world view instead fluxing flowing philosophy many type tradition anarchism exist mutually exclusive anarchist school thought differ fundamentally supporting anything extreme individualism complete collectivism strain anarchism often divided category social individualist anarchism similar dual classificati

In [76]:
print(df['SECTION_TEXT'].iloc[0])







'''anarchism''' is a political philosophy that advocates self-governed societies based on voluntary institutions. these are often described as stateless societies although several authors have defined them more specifically as institutions based on non-hierarchical free associations. anarchism holds the state to be undesirable unnecessary and harmful.

while anti-statism is central anarchism specifically entails opposing authority or hierarchical organisation in the conduct of all human relations including but not limited to the state system.  anarchism is usually considered an extreme left-wing ideology and much of anarchist economics and anarchist legal philosophy reflects anti-authoritarian interpretations of communism collectivism syndicalism mutualism or participatory economics.

anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy. many types and traditions of anarchism exist not all of which are m

## applying preprocessing 

In [3]:
df=pd.read_csv("/home/shahbaz/Desktop/A2/clean1.csv",index_col=False)

In [5]:
# Define the path for your clean CSV
csv_path = "/home/shahbaz/Desktop/A2/clean1.csv"
output_path = "/home/shahbaz/Desktop/A2/clean2.csv"

total_rows = sum(1 for row in open(csv_path, 'r')) - 1 

In [6]:
print(total_rows)

121201533


In [3]:
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'[^\w\s]', '', text)  
    
    # Tokenize and remove stop words and punctuation
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words and word not in punctuation]
    return " ".join(filtered_tokens)  # Joining tokens back into a string


In [9]:
chunk_size = 50000  # Adjust based on your memory constraints
temp_files = []  # To keep track of temp files



# Process the CSV in chunks
for i, chunk in enumerate(tqdm(pd.read_csv(csv_path, chunksize=chunk_size, index_col=False),total=int(total_rows / chunk_size), desc="Processing")):

    chunk = chunk.groupby('ARTICLE_ID')['SECTION_TEXT'].apply(lambda x: ' '.join(x)).reset_index()
    # Preprocess text directly
    chunk['SECTION_TEXT'] = chunk['SECTION_TEXT'].apply(preprocess_text)
    temp_file_path = f"/home/shahbaz/Desktop/A2/temp/temp_chunk_{i}.csv"
    chunk.to_csv(temp_file_path, index=False)
    temp_files.append(temp_file_path)



# The final preprocessed dataset is now saved in `output_path`

Processing:   4%|█                        | 100/2424 [33:19<12:54:33, 20.00s/it]


In [10]:
# Concatenate temp files into a final file
with open(output_path, 'w') as outfile:
    for fname in tqdm(temp_files):
        with open(fname) as infile:
            outfile.write(infile.read())
        os.remove(fname)  # Optionally, delete the temp file after merging

100%|█████████████████████████████████████████| 100/100 [00:31<00:00,  3.22it/s]


In [78]:
# import gc

# df=None
# gc.collect()
len(df)

537

# checking the processed dataset

In [48]:
# df = dd.read_csv("/home/shahbaz/Desktop/A2/clean1.csv", on_bad_lines='skip')

In [20]:
df=pd.read_csv("/home/shahbaz/Desktop/A2/clean2.csv",index_col=False)

In [25]:
df

,ARTICLE_ID,SECTION_TEXT
0,0,anarchism political philosophy advocates selfg...
1,1,autism neurodevelopmental disorder characteriz...
2,2,percentage diffusely reflected sunlight relati...
3,3,writing cursive forms named plural aes first l...
4,4,alabama state southeastern region united state...
...,...,...
817106,816926,anglian combination football league english fo...
817107,816927,national theatre oslo one norways largest prom...
817108,816928,midnight runner novel jack higgins published t...
817109,816929,thats call music may refer least two different...


In [15]:
df.isnull().sum()

ARTICLE_ID      0
SECTION_TEXT    0
dtype: int64

In [16]:
df=df.dropna()

In [26]:
print(df['SECTION_TEXT'].iloc[0])

anarchism political philosophy advocates selfgoverned societies based voluntary institutions often described stateless societies although several authors defined specifically institutions based nonhierarchical free associations anarchism holds state undesirable unnecessary harmful antistatism central anarchism specifically entails opposing authority hierarchical organisation conduct human relations including limited state system anarchism usually considered extreme leftwing ideology much anarchist economics anarchist legal philosophy reflects antiauthoritarian interpretations communism collectivism syndicalism mutualism participatory economics anarchism offer fixed body doctrine single particular world view instead fluxing flowing philosophy many types traditions anarchism exist mutually exclusive anarchist schools thought differ fundamentally supporting anything extreme individualism complete collectivism strains anarchism often divided categories social individualist anarchism simila

In [19]:
df.to_csv('/home/shahbaz/Desktop/A2/clean2.csv',index=False)

In [28]:
x=df.head(2000)

In [29]:
x.to_csv('/home/shahbaz/Desktop/A2/sample.csv',index=False)